## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBSSBSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
487
+-----------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|          coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+-----------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|8406.861268207338| 595.8026736315294|-2091.9443229657604|   2072.0|SBBSSBSB| 43.8567|-110.5889|       SNWD|0.11229294319897737|0.10783425804107764|0.052867242901368176|0.62769277

In [7]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [17]:
#extract longitude and latitude for each station
feature='coeff_3'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USC00486440,1970.0,-2091.944323
1,USC00480140,1964.0,344.660296
2,USC00486440,1950.0,-2071.509117
3,USC00486428,1968.0,294.743844
4,USC00486430,1955.0,-2239.019927


In [18]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,US1WYTT0009,USC00480140,USC00480603,USC00480605,USC00480778,USC00480865,USC00482054,USC00482242,USC00482375,USC00482725,...,USC00486440,USC00487260,USC00487760,USC00488209,USC00488858,USC00488870,USC00488875,USC00488880,USC00488884,USC00489305
year,,,,,,,,,,,,,,,,,,,,,
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,NaN,1406.722949,NaN,NaN,NaN,1604.540267,NaN,NaN,NaN,NaN,...,1432.085170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,589.081905
1950.0,NaN,-56.580676,NaN,NaN,NaN,-854.772868,NaN,NaN,NaN,NaN,...,-2071.509117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,-174.984349,NaN,NaN,NaN,338.925735,NaN,NaN,NaN,NaN,...,220.569100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952.0,NaN,306.735312,NaN,NaN,NaN,422.661768,NaN,NaN,NaN,NaN,...,411.870512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953.0,NaN,514.065505,NaN,NaN,NaN,1275.173283,NaN,NaN,NaN,NaN,...,-143.680173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,NaN,351.988768,NaN,846.599994,NaN,627.828992,NaN,NaN,NaN,NaN,...,717.366423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [19]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  1017.41864311
RMS removing mean-by-station=  936.612008356
RMS removing mean-by-year   =  635.847751658


In [16]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1122.69047274
0 after removing mean by year    = 702.159177023
0 after removing mean by stations= 601.039192384
1 after removing mean by year    = 595.368220259
1 after removing mean by stations= 594.540118584
2 after removing mean by year    = 594.353384834
2 after removing mean by stations= 594.30037449
3 after removing mean by year    = 594.283441102
3 after removing mean by stations= 594.277692132
4 after removing mean by year    = 594.275669506
4 after removing mean by stations= 594.274940397
